In [1]:
# DisulfideBond Playground
# Playing with the DisulfideBond class
# Author: Eric G. Suchanek, PhD.
# (c) 2022 Eric G. Suchanek, PhD., All Rights Reserved
# License: BSD
# Last Modification: 1/16/23
# Cα Cβ Sγ

# important preamble

import pandas as pd

import pyvista as pv
from pyvista import set_plot_theme

from Bio.PDB import *

# for using from the repo we 
from proteusPy import *
from proteusPy.data import *
from proteusPy.Disulfide import *


# override any default PDB globals
# location for PDB repository
PDB_ROOT = '/Users/egs/PDB/'

# location of cleaned PDB files - these are not stored in the repo
PDB_GOOD = '/Users/egs/PDB/good/'

# from within the repo 
PDB_REPO = '../pdb/'

# location of the compressed Disulfide .pkl files
MODELS = f'{PDB_ROOT}models/'

# pyvista setup for notebooks
pv.set_jupyter_backend('ipyvtklink')
set_plot_theme('document')


In [2]:
# Comment these out since they take so long.
# Download_Disulfides(pdb_home=PDB_ORIG, model_home=MODELS, reset=False)

#Extract_Disulfides(numb=1000, pdbdir=PDB_GOOD, datadir=MODELS, verbose=False, quiet=False)

PDB_SS = None
PDB_SS = DisulfideLoader(verbose=True)

ss_list = DisulfideList([], 'tmp')
PDB_SS.TotalDisulfides


Reading disulfides from: /Users/egs/repos/proteusPy/proteusPy/data/PDB_all_ss.pkl
Disulfides Read: 8210
Reading disulfide dict from: /Users/egs/repos/proteusPy/proteusPy/data/PDB_all_ss_dict.pkl
Reading Torsion DF /Users/egs/repos/proteusPy/proteusPy/data/PDB_all_SS_torsions.csv.
Read torsions DF.
PDB IDs parsed: 1000
Total Space Used: 1969317 bytes.


8210

In [3]:
ss0 = PDB_SS[0]
ss1 = PDB_SS[0]
ss2 = PDB_SS[1]
ss3 = PDB_SS[1]
sslist1 = DisulfideList([ss0, ss2], 'tmp')
sslist1.Torsion_RMS()

219.70970634935495

In [4]:
new = DisulfideList([], 'tmp')
sslist = PDB_SS.getlist()
sslist.min()


<Disulfide 1xr9_203A_259A SourceID: 1xr9 Proximal: 203 A Distal: 259 A>

In [5]:
sslist.max()

<Disulfide 1zjk_629A_660A SourceID: 1zjk Proximal: 629 A Distal: 660 A>

In [8]:

# one disulfide from the database
ss = Disulfide()
ss = PDB_SS[0]
#ss.pprint_all()

# get all disulfides for one structure. Make a 
# DisulfideList object to hold it
ss4yys = DisulfideList([], '4yys')
ss4yys = PDB_SS['4yys']

#ss4crn = DisulfideList([], '1crn')
#ss4crn = PDB_SS['1crn']

tot_ss = len(ss4yys) # number off ssbonds
print(f'tot {tot_ss}')


tot 6


In [9]:
print(f'{ss4yys.Torsion_RMS()}')

120.84848321514998


In [10]:
sslist = PDB_SS.SSList
tors_df = build_torsion_df(sslist)
tors_df.head()

100%|█████████████████████████████████████████████████████████| 8210/8210 [00:06<00:00, 1264.39it/s]


,source,ss_id,proximal,distal,chi1,chi2,chi3,chi4,chi5,energy,ca_distance,phi_prox,psi_prox,phi_dist,psi_dist
0,4yys,4yys_22A_65A,22,65,174.629233,82.517710,-83.322249,-62.523644,-73.827286,1.696237,4.502086,128.686790,128.686790,-105.731172,16.431694
1,4yys,4yys_56A_98A,56,98,-50.239063,-85.583916,97.275447,70.535692,179.046592,2.112566,4.967417,-30.489936,-30.489936,-59.378573,125.462589
2,4yys,4yys_156A_207A,156,207,62.598713,172.940042,-95.352637,-23.070934,-55.158480,2.331733,5.292317,150.741801,150.741801,60.457994,22.170381
3,4yys,4yys_22B_65B,22,65,173.666078,88.297996,-82.387276,-65.997032,-72.289506,1.958823,4.532387,130.848015,130.848015,-103.903213,16.298008
4,4yys,4yys_56B_98B,56,98,-56.410909,-81.401941,94.310784,67.035993,178.852441,1.444608,4.768629,-30.319989,-30.319989,-53.571262,120.853837


In [11]:
ss6fuf = PDB_SS['6fuf']


In [12]:
tors = pd.DataFrame()
tors = PDB_SS.getTorsions('')
tors

,source,ss_id,proximal,distal,chi1,chi2,chi3,chi4,chi5,energy,ca_distance,phi_prox,psi_prox,phi_dist,psi_dist
0,4yys,4yys_22A_65A,22,65,174.629233,82.517710,-83.322249,-62.523644,-73.827286,1.696237,4.502086,-149.776903,128.686790,-105.731172,16.431694
1,4yys,4yys_56A_98A,56,98,-50.239063,-85.583916,97.275447,70.535692,179.046592,2.112566,4.967417,-113.982916,-30.489936,-59.378573,125.462589
2,4yys,4yys_156A_207A,156,207,62.598713,172.940042,-95.352637,-23.070934,-55.158480,2.331733,5.292317,-156.930659,150.741801,60.457994,22.170381
3,4yys,4yys_22B_65B,22,65,173.666078,88.297996,-82.387276,-65.997032,-72.289506,1.958823,4.532387,-154.247515,130.848015,-103.903213,16.298008
4,4yys,4yys_56B_98B,56,98,-56.410909,-81.401941,94.310784,67.035993,178.852441,1.444608,4.768629,-113.329915,-30.319989,-53.571262,120.853837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8205,2p5r,2p5r_44B_92B,44,92,-124.400230,14.097260,87.928807,51.580778,-164.615423,6.941233,4.936201,-103.960336,93.039551,-52.651014,115.486532
8206,4qyz,4qyz_143A_253A,143,253,-69.050704,48.503674,150.647932,71.508814,-78.969552,7.378103,6.269383,-65.493849,-49.756331,-33.643753,-34.724290
8207,4qyz,4qyz_143A_250A,143,250,-69.050704,161.401761,-123.410979,-178.756423,170.219368,4.229833,7.803253,-65.493849,-49.756331,-75.614403,152.676285
8208,3tdj,3tdj_206A_261A,206,261,-43.122049,-78.811213,-89.985262,-53.676273,-50.840338,2.058045,5.446051,46.516562,39.500316,-114.399870,102.881614


In [13]:
ss1 = ss4yys.get_by_name('4yys_22A_65A')
ss1

<Disulfide 4yys_22A_65A SourceID: 4yys Proximal: 22 A Distal: 65 A>

In [14]:
Check_chains('4yys', PDB_GOOD)

ssbond dict: {1: ('22', '65', 'A', 'A'), 2: ('56', '98', 'A', 'A'), 3: ('156', '207', 'A', 'A'), 4: ('22', '65', 'B', 'B'), 5: ('56', '98', 'B', 'B'), 6: ('156', '207', 'B', 'B')}
multiple chains. [<Chain id=A>, <Chain id=B>]
Chain: A, length: 504
Chain: B, length: 454
chain lengths are unequal: [504, 454]


False

In [15]:
ss4yys_a = ss4yys.by_chain('A')
ss4yys_a.Torsion_RMS()


171.30285555734537

In [16]:
ss4yys_b = ss4yys.by_chain('B')
ss4yys_b.Torsion_RMS()

170.49799106566022

In [18]:
ss4yys_a1 = ss4yys_a[0]
# print(ss4yys_a1.repr_ss_coords())


In [19]:
ss4yys_b1 = ss4yys_b[0]
ss4yys_b1

<Disulfide 4yys_22B_65B SourceID: 4yys Proximal: 22 B Distal: 65 B>

In [20]:
ss4yys_a1.Distance_RMS(ss4yys_b1)

0.1145026461405659

In [21]:
ss4yys_a1.Torsion_RMS(ss4yys_b1)

7.045760800877229

In [ ]:
ss4yys_a1 == ss4yys_b1

In [ ]:
chns = ss4yys.get_chains()
ss4yys.has_chain('yyy')
chns

In [ ]:
# load SS bonds by PDB ID
ss1 = PDB_SS['4yys']
print(ss1)
print(ss1[0].get_full_id())


In [ ]:
# you can loop over the IDList list and extract by ID
#
for id in PDB_SS.IDList[:5]:    # just show last 5
    # get the SS bonds for the given ID
    ssb = PDB_SS[id]
    numb_ss = len(ssb)
    print(f'ID: {id} has {numb_ss} Disulfides:')
    for bond in ssb:
        print(bond)
    print('\n')
    

In [ ]:
ss_list = PDB_SS.getlist()
ss0 = ss_list[0]

print(ss0.proximal_residue_fullid)
print(ss0.chi3)
len(ss_list)


In [ ]:
# routine creates 2 lists  for left-handed and right-handed disulfides 
ss_list = PDB_SS.getlist()
left_handed = DisulfideList([], 'left_handed')
right_handed = DisulfideList([], 'right_handed')

i = 0

for i in range(len(ss_list)):
    ss = ss_list[i]
    if ss.chi3 < 0:
        left_handed.append(ss)
    else:
        right_handed.append(ss)


print(f'Left Handed: {len(left_handed)}, Right Handed: {len(right_handed)}')



In [ ]:
from proteusPy.Disulfide import Disulfide

# make some empty disulfides
ss1 = Disulfide('ss1')
ss2 = Disulfide('ss2')

# make a DisulfideList containing ss1, named 'tmp'
sslist = DisulfideList([ss1], 'tmp')
sslist.append(ss2)

# load the PDB Disulfide database
PDB_SS = None
PDB_SS = DisulfideLoader(verbose=True)

# extract a disulfide with typical index
ss1 = PDB_SS[0]
#print(f'{ss1.pprint_all()}')

# grab a subset via slicing
subset = DisulfideList(PDB_SS[0:10],'subset')

In [ ]:
rms = subset.Torsion_RMS()
rms

In [ ]:
torsions = PDB_SS.getTorsions()
torsions.sort_values(by=['energy'], ascending=False, inplace=True)

torsions.head(10)

In [ ]:
idx_max = int(torsions['energy'].idxmax())
print(f'IDMAX: {idx_max}')
ssmax = PDB_SS[idx_max]
ssmax

In [ ]:
toget = 200

badlist = torsions['ss_id']

bad_SS_list = DisulfideList([],'20 top high energy')
ss = Disulfide()

for i in range(toget):
    ssid = torsions.iloc[i]['ss_id']
    ss = PDB_SS.get_by_name(ssid)
    #print(f'ID: {ss.name}: {ss.energy:.2f} kcal/mol CA: {ss.ca_distance:.2f}')
    bad_SS_list.append(ss)



In [ ]:

tors_bad_rms = bad_SS_list.Torsion_RMS()
tors_bad_rms

In [ ]:
goodlist = torsions['ss_id']

good_SS_list = DisulfideList([],'20 top low energy')
ss = Disulfide()

for i in range(toget):
    if i == 0:
        continue
    
    ssid = torsions.iloc[-i]['ss_id']
    ss = PDB_SS.get_by_name(ssid)
    #print(f'ID: {ss.name}: {ss.energy:.2f} kcal/mol CA: {ss.ca_distance:.2f}')
    good_SS_list.append(ss)


In [ ]:

tors_good_rms = good_SS_list.Torsion_RMS()
tors_good_rms

In [ ]:
from scipy.spatial import distance_matrix

good_array = good_SS_list.get_torsion_array()
bad_array = bad_SS_list.get_torsion_array()
good_array.shape

In [ ]:

dm1 = distance_matrix(good_array, good_array)
dm2 = distance_matrix(bad_array, bad_array)



In [ ]:
import numpy as np

def EDM(A, B):
    p1 = np.sum(A**2, axis=1)[:, np.newaxis]
    p2 = np.sum(B**2, axis=1)
    p3 = -2 * np.dot(A, B.T)
    res = p1 + p2 + p3
    #res = numpy.sqrt(res)
    return res


In [ ]:
dm3 = np.sqrt(dm2)

In [ ]:
import plotly_express as px
fig = px.imshow(dm1)
fig.show()

In [ ]:
import plotly_express as px
fig = px.imshow(dm2)
fig.show()

In [ ]:
df = pd.DataFrame(dm1)
df.head()